In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.gaussian_process as gp
from sklearn.neural_network import MLPRegressor
from scipy.interpolate import UnivariateSpline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.signal import savgol_filter

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from PredictXANES import xanes_derivatives

In [7]:
def split_data(df, size_list):
    '''
    The spectra are split into a training set and 
    testing set with a ratio of 4:1. The predicted features are the coordination number, 
    number of Cu atoms, and number of Te atoms.
    '''
    
    X = df.drop(labels=['CN', 'Num Cu', 'Num Te'], axis=1)
    list = X.columns.tolist()

    X.columns = range(X.shape[1])
    X.columns = list
   
    y = df[['CN', 'Num Cu', 'Num Te']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, size_list, random_state=12)
    
    return X_train, X_test, y_train, y_test, X, y

def derivatives(X):
    '''
    derivatives returns the first, second, and combined first and second derivitive dataframes
    '''
    d1, d2 = xanes_derivatives.xanes_derivatives(X)
    df1 = pd.DataFrame(d1)
    df2 = pd.DataFrame(d2)
    df3 = pd.concat([df1, df2], axis=1)

    return df1, df2, df3
    
def train(X_train, y_train):
    '''
    train_layer trains the neural network. One layer is added, and the depth of the that layer is optimized by 
    looking at the loss from a range of 1 to 100, the maximum number of features being trained on.
    '''
    
    loss = []
    for i in range(100):
        if i == 0:
            pass
        else:
            nn = MLPRegressor(hidden_layer_sizes=(i), activation='identity', solver='lbfgs', max_iter=2000, random_state=28)
            nn = nn.fit(X_train, y_train)
            loss.append(nn.loss_)

    lossdf = pd.DataFrame(loss)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(lossdf)
    return lossdf, nn

def analyse_layer(X_train, X_test, y_train, y_test, nn):
    '''
    analyse_layer returns the training mse and r2 values for the layer in the neural network
    '''
    
    y_train_pred = nn.predict(X_train)
    y_test_pred = nn.predict(X_test)
    train_score_CN = r2_score(y_train.iloc[:,0], y_train_pred[:,0])
    train_score_Cu = r2_score(y_train.iloc[:,1], y_train_pred[:,1]) 
    train_score_Te = r2_score(y_train.iloc[:,2], y_train_pred[:,2])
    test_score_CN = r2_score(y_test.iloc[:,0], y_test_pred[:,0])
    test_score_Cu = r2_score(y_test.iloc[:,1], y_test_pred[:,1])
    test_score_Te = r2_score(y_test.iloc[:,2], y_test_pred[:,2])
    train_mse_CN = mean_squared_error(y_train.iloc[:,0], y_train_pred[:,0])
    train_mse_Cu = mean_squared_error(y_train.iloc[:,1], y_train_pred[:,1])
    train_mse_Te = mean_squared_error(y_train.iloc[:,2], y_train_pred[:,2])
    test_mse_CN = mean_squared_error(y_test.iloc[:,0], y_test_pred[:,0])
    test_mse_Cu = mean_squared_error(y_test.iloc[:,1], y_test_pred[:,1])
    test_mse_Te = mean_squared_error(y_test.iloc[:,2], y_test_pred[:,2])

    train_score_list = [train_score_CN, train_score_Cu, train_score_Te]
    train_mse_list = [train_mse_CN, train_mse_Cu, train_mse_Te]
    test_mse_list = [test_mse_CN, train_mse_Cu, train_mse_Te]
    test_score_list = [test_score_CN, test_score_Cu, test_score_Te]
    print('training mse =  '+ str(train_mse_list))
    print('testing mse = ' + str(test_mse_list))
    print('training R2 = ' + str(train_score_list))
    print('testing R2 = ' + str(test_score_list))
    
    return test_mse_list, test_score_list, y_test_pred

def append_layer_to_list(test_mse_list, test_score_list, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list):
    '''
    append_layer_to_list takes the mse and score lists and appends them to a list for plotting erros vs noise, 
    note that here mse is converted to rmse
    '''
    
    rmse_CN = np.sqrt(test_mse_list[0])
    rmse_CN_master_list.append(rmse_CN)
    rmse_Cu = np.sqrt(test_mse_list[1])
    rmse_Cu_master_list.append(rmse_Cu)
    rmse_Te = np.sqrt(test_mse_list[2])
    rmse_Te_master_list.append(rmse_Te)
    
    score_CN = test_score_list[0]
    score_CN_master_list.append(score_CN)
    score_Cu = test_score_list[1]
    score_Cu_master_list.append(score_Cu)
    score_Te = test_score_list[2]
    score_Te_master_list.append(score_Te)

def plot_parity(y_test, y_test_pred):
    '''
    plot_parity is a function to generate parity plots for the layer to check performance for predicting 
    coordination number, number of nearest Te atoms, and number of nearest Cu atoms.
    '''
    x1 = np.linspace(8, 12, 50)
    x2 = np.linspace(4, 8, 50)
    x3 = np.linspace(4, 5, 50)

    plt.figure(figsize=[14,4])
    plt.subplots_adjust(wspace=0.3)
    ax1 = plt.subplot(1,3,1)
    ax1.scatter(y_test.iloc[:,0], y_test_pred[:,0])
    ax1.plot(x1, x1, color='red')
    ax1.set_xlabel('True CN')
    ax1.set_ylabel('Pred CN')
    ax1.set_title('Test CN')
    plt.axis('equal')

    ax2 = plt.subplot(1,3,2)
    ax2.scatter(y_test.iloc[:,1], y_test_pred[:,1])
    ax2.plot(x2, x2, color='red')
    ax2.set_xlabel('True Cu Num')
    ax2.set_ylabel('Pred Cu Num')
    ax2.set_title('Test Cu Num')
    plt.axis('equal')


    ax3 = plt.subplot(1,3,3)
    ax3.scatter(y_test.iloc[:,2], y_test_pred[:,2])
    ax3.plot(x3, x3, color='red')
    ax3.set_xlabel('True Te Num')
    ax3.set_ylabel('Pred Te Num')
    ax3.set_title('Test Te Num')
    plt.axis('equal')
    
    
def run_layer(X_train, X_test, y_train, y_test, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list):
    '''
    run_layer is a wrapper function that executes the training, testing, analysis, and plotting for a 
    single layer of the neural network
    '''
    lossdf, nn = train(X_train, y_train)
    test_mse_list, test_score_list, y_test_pred = analyse_layer(X_train, X_test, y_train, y_test, nn) #series from dataframe
    append_layer_to_list(test_mse_list, test_score_list, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list)
    plot_parity(y_test, y_test_pred)

def MLP_per_set_size(df, size_list, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list):
    '''
    MLP_per_noise is a wrapper function that runs a layer of neural network per level of noise for spectrum
    '''
    for entry in size_list:
        print('set size: ', entry)
        X_train, X_test, y_train, y_test, X, y = split_data(df, size_list)
        run_layer(X_train, X_test, y_train, y_test, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list)
    print(rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list)

def MLP_per_set_size_1(df, size_list, rmse_CN_master_list_1, score_CN_master_list_1, rmse_Cu_master_list_1, score_Cu_master_list_1, rmse_Te_master_list_1, score_Te_master_list_1):
    '''
    MLP_per_noise_1 is a wrapper function that runs a layer of neural network per level of noise for the first derivative
    '''
    for entry in size_list:
        print('set size: ', entry)
        _, _, _, _, X, y = split_data(df, size_list)
        df1, df2, df3, = derivatives(X)
        X_train, X_test, y_train, y_test = train_test_split(df1, y)
        run_layer(X_train, X_test, y_train, y_test, rmse_CN_master_list_1, score_CN_master_list_1, rmse_Cu_master_list_1, score_Cu_master_list_1, rmse_Te_master_list_1, score_Te_master_list_1)
    print(rmse_CN_master_list_1, score_CN_master_list_1, rmse_Cu_master_list_1, score_Cu_master_list_1, rmse_Te_master_list_1, score_Te_master_list_1)

def MLP_per_set_size_2(df, size_list, rmse_CN_master_list_2, score_CN_master_list_2, rmse_Cu_master_list_2, score_Cu_master_list_2, rmse_Te_master_list_2, score_Te_master_list_2):
    '''
    MLP_per_noise is a wrapper function that runs a layer of neural network per level of noise for the second derivative
    '''
    for entry in size_list:
        print('set size: ', entry)
        _, _, _, _, X, y = split_data(df, size_list)
        df1, df2, df3, = derivatives(X)
        X_train, X_test, y_train, y_test = train_test_split(df2, y)
        run_layer(X_train, X_test, y_train, y_test, rmse_CN_master_list_2, score_CN_master_list_2, rmse_Cu_master_list_2, score_Cu_master_list_2, rmse_Te_master_list_2, score_Te_master_list_2)
    print(rmse_CN_master_list_2, score_CN_master_list_2, rmse_Cu_master_list_2, score_Cu_master_list_2, rmse_Te_master_list_2, score_Te_master_list_2)
    
    
def MLP_per_set_size_1_2(df, size_list, rmse_CN_master_list_1_2, score_CN_master_list_1_2, rmse_Cu_master_list_1_2, score_Cu_master_list_1_2, rmse_Te_master_list_1_2, score_Te_master_list_1_2):
    '''
    MLP_per_noise is a wrapper function that runs a layer of neural network per level of noise for both the first and second derivative
    '''
    for entry in size_list:
        print('set size: ', entry)
        _, _, _, _, X, y = split_data(df, size_list)
        df1, df2, df3, = derivatives(X)
        X_train, X_test, y_train, y_test = train_test_split(df3, y)
        run_layer(X_train, X_test, y_train, y_test, rmse_CN_master_list_1_2, score_CN_master_list_1_2, rmse_Cu_master_list_1_2, score_Cu_master_list_1_2, rmse_Te_master_list_1_2, score_Te_master_list_1_2)
    print(rmse_CN_master_list_1_2, score_CN_master_list_1_2, rmse_Cu_master_list_1_2, score_Cu_master_list_1_2, rmse_Te_master_list_1_2, score_Te_master_list_1_2)


Data import of 10,000 calculated average XANES spectra along with averaged coordination numbers, number of Cu atoms within 3 angstroms, and number of Te atoms within 3 angstroms.

In [4]:
df = pd.read_csv('mu_cn10000.csv')

## Spectra Only:

Create lists for plotting noise vs R2 an RMSE

In [5]:
size_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
rmse_CN_master_list = []
score_CN_master_list = []
rmse_Cu_master_list = []
score_Cu_master_list = []
rmse_Te_master_list = []
score_Te_master_list = []

In [8]:
MLP_per_set_size(df, size_list, rmse_CN_master_list, score_CN_master_list, rmse_Cu_master_list, score_Cu_master_list, rmse_Te_master_list, score_Te_master_list)

set size:  0.05


ValueError: Found input variables with inconsistent numbers of samples: [10000, 10000, 9]

# Repeated for training with first derivative.

In [ ]:
size_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
rmse_CN_master_list_1 = []
score_CN_master_list_1 = []
rmse_Cu_master_list_1 = []
score_Cu_master_list_1 = []
rmse_Te_master_list_1 = []
score_Te_master_list_1 = []

In [ ]:
MLP_per_set_size_1(df, size_list, rmse_CN_master_list_1, score_CN_master_list_1, rmse_Cu_master_list_1, score_Cu_master_list_1, rmse_Te_master_list_1, score_Te_master_list_1)

# Repeated for training with both the first and second derivative.

In [ ]:
size_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
rmse_CN_master_list_1_2 = []
score_CN_master_list_1_2 = []
rmse_Cu_master_list_1_2 = []
score_Cu_master_list_1_2 = []
rmse_Te_master_list_1_2 = []
score_Te_master_list_1_2 = []

In [ ]:
MLP_per_set_size_1_2(df, size_list, rmse_CN_master_list_1_2, score_CN_master_list_1_2, rmse_Cu_master_list_1_2, score_Cu_master_list_1_2, rmse_Te_master_list_1_2, score_Te_master_list_1_2)

In [ ]:
print(rmse_CN_master_list_1_2)

In [ ]:
size_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
rmse_CN_master_list_2 = []
score_CN_master_list_2 = []
rmse_Cu_master_list_2 = []
score_Cu_master_list_2 = []
rmse_Te_master_list_2 = []
score_Te_master_list_2 = []

In [ ]:
MLP_per_set_size_2(df, size_list, rmse_CN_master_list_2, score_CN_master_list_2, rmse_Cu_master_list_2, score_Cu_master_list_2, rmse_Te_master_list_2, score_Te_master_list_2)

In [ ]:
print(rmse_CN_master_list_2)

In [ ]:
plt.rcParams.update({'font.size': 14})
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12,5])
plt.subplots_adjust(wspace=0.3)
ax1.plot(size_list, score_CN_master_list, c='blue', lw=2.5, label='Spectrum')
ax1.plot(size_list, score_CN_master_list_1, c='red', lw=2.5, label='First Derivative')
ax1.plot(size_list, score_CN_master_list_2, c='darkorange', lw=2.5, label='Second Derivative')
ax1.plot(size_list, score_CN_master_list_1_2, c='green', lw=2.5, label='Both Derivatives')
ax1.set_xlabel('Set Size', fontsize=16)
ax1.set_ylabel('$R^2$', fontsize=16)
ax2.plot(size_list, rmse_CN_master_list, c='blue', lw=2.5)
ax2.plot(size_list, rmse_CN_master_list_1, c='red', lw=2.5)
ax2.plot(size_list, rmse_CN_master_list_2, c='darkorange', lw=2.5)
ax2.plot(size_list, rmse_CN_master_list_1_2, c='green', lw=2.5)
ax2.set_xlabel('Set Size', fontsize=16)
ax2.set_ylabel('RMSE', fontsize=16)
ax1.legend()
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 14})
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12,5])
plt.subplots_adjust(wspace=0.3)
ax1.plot(size_list, score_Cu_master_list, c='blue', lw=2.5, label='Spectrum')
ax1.plot(size_list, score_Cu_master_list_1, c='red', lw=2.5, label='First Derivative')
ax1.plot(size_list, score_Cu_master_list_2, c='darkorange', lw=2.5, label='Second Derivative')
ax1.plot(size_list, score_Cu_master_list_1_2, c='green', lw=2.5, label='Both Derivatives')
ax1.set_xlabel('Set Size', fontsize=16)
ax1.set_ylabel('$R^2$', fontsize=16)
ax2.plot(size_list, rmse_Cu_master_list, c='blue', lw=2.5)
ax2.plot(size_list, rmse_Cu_master_list_1, c='red', lw=2.5)
ax2.plot(size_list, rmse_Cu_master_list_2, c='darkorange', lw=2.5)
ax2.plot(size_list, rmse_Cu_master_list_1_2, c='green', lw=2.5)
ax2.set_xlabel('Set Size', fontsize=16)
ax2.set_ylabel('RMSE', fontsize=16)
ax1.legend()
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 14})
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12,5])
plt.subplots_adjust(wspace=0.3)
ax1.plot(size_list, score_Te_master_list, c='blue', lw=2.5, label='Spectrum')
ax1.plot(size_list, score_Te_master_list_1, c='red', lw=2.5, label='First Derivative')
ax1.plot(size_list, score_Te_master_list_2, c='darkorange', lw=2.5, label='Second Derivative')
ax1.plot(size_list, score_Te_master_list_1_2, c='green', lw=2.5, label='Both Derivatives')
ax1.set_xlabel('Set Size', fontsize=16)
ax1.set_ylabel('$R^2$', fontsize=16)
ax2.plot(size_list, rmse_Te_master_list, c='blue', lw=2.5)
ax2.plot(size_list, rmse_Te_master_list_1, c='red', lw=2.5)
ax2.plot(size_list, rmse_Te_master_list_2, c='darkorange', lw=2.5)
ax2.plot(size_list, rmse_Te_master_list_1_2, c='green', lw=2.5)
ax2.set_xlabel('Set Size', fontsize=16)
ax2.set_ylabel('RMSE', fontsize=16)
ax1.legend()
plt.show()